In [ ]:
import pandas as pd
import requests
import json
import datetime
import concurrent.futures
import swifter

In [ ]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

In [ ]:
processos = []

def lista_para_dataframe(dados_dict):
  for processo in dados_dict['hits']['hits']:
    numero_processo = processo['_source']['numeroProcesso']
    grau = processo['_source']['grau']
    classe = processo['_source']['classe']['nome']
    try:
      assuntos = processo['_source']['assuntos'] # Pode ter mais de um
    except:
      assuntos = []
    data_ajuizamento = processo['_source']['dataAjuizamento']
    ultima_atualizacao = processo['_source']['dataHoraUltimaAtualizacao']
    #formato = processo['_source']['formato']['nome']
    codigo = processo['_source']['orgaoJulgador']['codigo']
    orgao_julgador = processo['_source']['orgaoJulgador']['nome']
    municipio = processo['_source']['orgaoJulgador']['codigoMunicipioIBGE']
    sort = processo['sort'][0]
    try:
      movimentos = processo['_source']['movimentos']
    except:
      movimentos = []

    processos.append([numero_processo, classe, data_ajuizamento, ultima_atualizacao, \
                      codigo, orgao_julgador, municipio, grau, assuntos, movimentos, sort])

  df = pd.DataFrame(processos, columns=['numero_processo', 'classe', 'data_ajuizamento', 'ultima_atualizacao', \
                        'codigo', 'orgao_julgador', 'municipio', 'grau', 'assuntos', 'movimentos', 'sort'])
  #ddf = dd.from_pandas(df)
  #ddf.sample(frac=0.001)
  df.sample(5)
  return df

In [ ]:
df_tjrn = pd.DataFrame()
url = "https://api-publica.datajud.cnj.jus.br/api_publica_tjrn/_search"
api_key = "APIKey cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw==" # Chave pública
tribunal = 'TJRN'
grau = 'JE'
orgaoJulgador = '4º JUIZADO ESPECIAL DA FAZENDA PÚBLICA'
size = 1000
data = "2024-01-01"

payload = json.dumps(
{
 "size": size,
 "query": {
    "bool": {
      "must": [
          {"match": {"tribunal": tribunal}},
          {"match": {"orgaoJulgador.nome": orgaoJulgador}},
          {"match": {"grau": grau}},
          {"range": {"dataAjuizamento": {"gte": data }}}
      ]
    }
},
  "sort": [{"@timestamp": {"order": "asc"}}]
})

headers = {
  'Authorization': api_key,
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)  # <Response [200]>
dados_dict = response.json() # <class 'dict'>
df_tjrn = lista_para_dataframe(dados_dict)
numero_processos = size

while numero_processos == size:
  numero_processos = len(dados_dict['hits']['hits'])
  ultima_posicao_dicionario = dados_dict['hits']['hits'][(len(dados_dict['hits']['hits'])-1)]['sort'][0]
  #print(f'Partindo da posição: {ultima_posicao_dicionario}')
  payload = json.dumps(
  {
  "size": size,
  "query": {
      "bool": {
        "must": [
          {"match": {"tribunal": tribunal}},
          {"match": {"orgaoJulgador.nome": orgaoJulgador}},
          {"match": {"grau": grau}},
          {"range": {"dataAjuizamento": {"gte": data}}}
          
        ]
      }
  },
    "search_after": [ ultima_posicao_dicionario ],
    "sort": [{"@timestamp": {"order": "asc"}}]
  })

  headers = {
    'Authorization': api_key,
    'Content-Type': 'application/json'
  }

  response = requests.request("POST", url, headers=headers, data=payload)  # <Response [200]>
  dados_dict = response.json() # <class 'dict'>
  numero_processos = len(dados_dict['hits']['hits'])
  ultima_posicao_dicionario = dados_dict['hits']['hits'][(len(dados_dict['hits']['hits'])-1)]['sort']
  df_tjrn = pd.concat([df_tjrn, lista_para_dataframe(dados_dict)])
  tamanho_dataset = len(df_tjrn.index)
  #now = datetime.datetime.now()
  ultima_data_ajuizamento = dados_dict['hits']['hits'][len(dados_dict['hits']['hits'])-1]['_source']['dataAjuizamento']
  print(f'{datetime.datetime.now()} \tNúmero de processos: {tamanho_dataset} \t Data do último processo adicionado: {ultima_data_ajuizamento}' )
  #if tamanho_dataset > 2000000:
  #  break

print(f'Número de processos incorporados: {tamanho_dataset}')

In [ ]:
# Descobrindo código e órgão julgador
if len(dados_dict) > 1:
  codigo = dados_dict['hits']['hits'][0]['_source']['orgaoJulgador']['codigo']
  orgao_julgador = dados_dict['hits']['hits'][0]['_source']['orgaoJulgador']['nome']
  data_ajuizamento = dados_dict['hits']['hits'][0]['_source']['dataAjuizamento']
  dataHoraUltimaAtualizacao = dados_dict['hits']['hits'][0]['_source']['dataHoraUltimaAtualizacao']

print (f'Código: {codigo} - Órgão julgador: {orgao_julgador} - Data de Ajuizamento: {data_ajuizamento} - Data da Última Atualização: {dataHoraUltimaAtualizacao}')

In [ ]:
def converte_data(data_str):
    return pd.to_datetime(data_str).tz_convert('America/Sao_Paulo')


def gera_lista_assuntos(assuntos_do_df):
    lst_assuntos=[]
    for assunto in assuntos_do_df:
        try:
            lst_assuntos.append(assunto.get('nome'))
        except:
            lst_assuntos.append('')

    return lst_assuntos


def gera_lista_movimentos(movimentos):
    lst_movimentos_final =[]
    for movimento in movimentos:
        codigo = movimento.get('codigo')
        nome = movimento.get('nome')
        data_hora = movimento.get('dataHora')
        if data_hora:
            data_hora = converte_data(data_hora)
        lst_movimentos_final.append([ codigo, nome, data_hora])
    return lst_movimentos_final

def calcula_primeira_sentenca(movimentos):
    lst_tempo_decorrido = []
    for movimento in movimentos:
        data_inicial = pd.datetime()
        primeira_sentenca = 'False'
        if movimento.get('codigo') == 26: # Distribuição
            data_inicial = movimento.get('dataHora')
        if movimento.get('codigo') == 193: # Julgamento
            lst_tempo_decorrido.append(movimento.get('dataHora') - data_inicial)
            
    return lst_tempo_decorrido

In [ ]:
def process_movimento(movimento):
    codigo = movimento.get('codigo')
    nome = movimento.get('nome')
    data_hora = movimento.get('dataHora')
    if data_hora:
        data_hora = converte_data(data_hora)
    return [codigo, nome, data_hora]

def gera_lista_movimentos_multithread(movimentos):
    lst_movimentos_final = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(executor.map(process_movimento, movimentos))
        lst_movimentos_final.extend(results)
    return lst_movimentos_final

In [ ]:
df = df_tjrn
df.reset_index(drop=True, inplace=True)

In [ ]:
df['assuntos'] = df['assuntos'].swifter.apply(gera_lista_assuntos)
df['movimentos'] = df['movimentos'].swifter.apply(gera_lista_movimentos_multithread)
df['data_ajuizamento'] = df['data_ajuizamento'].swifter.apply(converte_data)
df['ultima_atualizacao'] = df['ultima_atualizacao'].swifter.apply(converte_data)
try:
  df['movimentos']= df['movimentos'].swifter.apply(lambda x: sorted(x, key=lambda tup: tup[2], reverse=False))
except:
  pass
df.sample(5)

In [ ]:
nome_movimentacoes = list()
codigo_movimentacoes = list()

  
for movimentacao in df['movimentos']:
  for i in range(0,len(movimentacao)):
    for j in range(0,len(movimentacao[i])):
      #print(movimentacao[i][j])
      if type(movimentacao[i][j]) == str:
        if movimentacao[i][j] not in nome_movimentacoes:
          nome_movimentacoes.append(movimentacao[i][j])
      if type(movimentacao[i][j]) == int:
        if movimentacao[i][j] not in codigo_movimentacoes:
          codigo_movimentacoes.append(movimentacao[i][j])

In [ ]:
print(df.head(5))

In [ ]:
print(len(nome_movimentacoes))
print(len(codigo_movimentacoes))

In [ ]:
for mov in nome_movimentacoes:
  print(mov)

In [ ]:
print(df['orgao_julgador'].unique())

In [ ]:
#df.to_parquet('tjrn_je.parquet', compression='gzip')
nome_df = tribunal + '_' + orgaoJulgador + '_' + data + '.csv'
df.to_csv(nome_df, sep=';', header=True, index=False, compression='zip')